1. Physics-Driven Framework

Key Physics for CBN-GCr15 Interaction

Brittle-Ductile Transition: GCr15 steel under CBN grinding exhibits mixed-mode material removal. 

Critical grinding depth $a_{p,\text{crit}} \approx 30\ \mu m$ (empirical for bearing steels) separates brittle fracture (low $a_p$) from plastic flow (high $a_p$).


Specific Grinding Energy:
$$
e_c = \frac{F_t \cdot v_s}{a_p \cdot v_w \cdot w} \quad (\textrm{J/mm})^3
$$

(F_t) (tangential force) can be approximated via CBN-specific empirical model:
$$
F_t = k \cdot a_p^{0.8} \cdot v_s^{-0.2} \quad 
$$
$ k = 1.2\ \text{N} /\mu m^{0.8}$ for GCr15

Derived from [Marinescu et al., Handbook of Machining with Grinding Wheels, 2016]
Thermal Damage Risk:
$$
T_{\text{max}} = \frac{q \cdot a_p^{0.5}}{K \cdot \sqrt{\pi \cdot \kappa}}, \quad \text{where } \kappa = \frac{K}{\rho C_p} \text{ (thermal diffusivity)}
$$

For GCr15: $K = 46\ \text{W/mK}, \rho C_p = 3.8\ \text{J/cm³K}$.

2. Feature Engineering Pipeline
A. Physics-Based Features
Brittle-Ductile Indicator:
$$
\text{BDI} = \frac{a_p}{a_{p,\text{crit}}} \quad \text{(>1 implies ductile-dominated)}
$$
Adjusted Specific Energy:
$$
e_c' = e_c \cdot \left(1 + 0.1 \cdot \frac{v_w}{40}\right) \quad \text{(accounts for workpiece speed nonlinearity)}
$$
Thermal Severity:
$$
S_t = \frac{T_{\text{max}}}{T_{\text{material}}} \quad ((T_{\text{material}} = 800\circ)C for GCr15 tempering threshold)
$$
B. Sensor Signal Features
AE (4 MHz sampling):

- Narrowband (50–300 kHz):
    - Fracture events (spalling in GCr15) → Burst AE energy.
    - Extract: Wavelet energy in 150–250 kHz band, burst count/sec.
- Broadband (1–3 MHz):
    - Grain wear/microfracture → Continuous AE RMS.
- Vibration (51.2 kHz sampling):
    - Spectral Peaks at $f_{\text{mesh}} = \frac{v_s \cdot N_{\text{grits}}}{\pi \cdot D_w}$ (≈400–800 Hz for CBN wheel).
    - Envelope Analysis of 2–5 kHz band to detect wheel imbalance (critical for thin-walled bushing stability).

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

def compute_ec(v_s, v_w, a_p, w=214):  # w = workpiece OD (mm)
    """Compute adjusted specific energy with CBN-GCr15 empirical model."""
    # Empirical tangential force (Marinescu model for bearing steel)
    F_t = 1.2 * (a_p**0.8) * (v_s**-0.2)  # F_t in Newtons
    e_c = (F_t * v_s) / (a_p * v_w * w)
    
    # Nonlinear adjustment for v_w (calibrated to your 20-80 rpm range)
    v_w_ref = interp1d([20, 80], [0.85, 1.15])(v_w)  # Scaling factor
    return e_c * v_w_ref

def compute_bdi(a_p):
    """GCr15-specific transition threshold (30µm critical depth)."""
    return a_p / 30.0  # Returns ratio relative to critical depth

def compute_st(v_s, a_p, grinding_time):
    """Max temperature ratio (T_max/T_material) for GCr15."""
    K = 46e-3  # W/mmK
    rho_cp = 3.8e-3  # J/mm³K
    q = 0.8 * v_s * compute_ec(v_s, 40, a_p)  # Heat flux approximation
    
    def _temp_ode(t, T):
        return (q * np.sqrt(a_p)) / (K * np.sqrt(np.pi * (K/rho_cp) * t))
    
    sol = solve_ivp(_temp_ode, [1e-6, grinding_time], [25], method='Radau')
    T_max = sol.y[0][-1]
    return T_max / 800.0  # 800°C = GCr15 tempering threshold


IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed